# NBA Player Data Analysis Project

## Project Outline

1. **Introduction**
   - Overview of the project
   - Goals and objectives

2. **Data Collection**
   - Import necessary libraries
   - Fetch player data from the NBA API

3. **Data Processing**
   - Load data into a Pandas DataFrame
   - Data cleaning and transformation

4. **Data Analysis**
   - Exploratory data analysis (EDA)
   - Visualizations

5. **Conclusion**
   - Summary of findings
   - Future work


## 0. Environment

### Python

This project is written in Python, which means that Python must be installed in your environment to run the project. The minimum supported version is 3.10.

#### Windows

You can use the Windows package manager `winget`, or the [installer](https://www.python.org/downloads/windows/) from the website.
```powershell
# you can change the version in the package name to your desired version
winget install Python.Python.3.12
```

#### MacOS
Python is already installed by default on recent versions of MacOS. If you have an older version that is not supported, you can use the [Homebrew](https://brew.sh/) package manager to install it, or the [installer](https://www.python.org/downloads/macos/) from the website.
```zsh
brew install python
```

#### Linux
Python is already installed by default on most distributions of Linux. If it isn't, you can use your distribution's package manager to install Python.

### Virtual Environment

It's generally recommended that you use a virtual environment (or venv) for this project. That way, all dependencies can be installed for the project without affecting the rest of your system. You can create a venv with Python:

```bash
python -m venv .venv
```

To activate the virtual environment in your shell, you can use the following commands.

On Windows:

```powershell
.venv\Scripts\activate
```

On other operating systems:

```bash
.venv/bin/activate
```

### Dependencies

This project uses [Poetry](https://python-poetry.org/) to manage its dependencies. You can install the dependencies with the `poetry` command:

`poetry install`

If you don't want to use Poetry, a `requirements.txt` is also provided. You can install this using `pip`:

`pip install -r requirements.txt`

### Imports

In [ ]:
import json
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from typing import Dict

### Environment Variables

We will load all our environment variables from a `.env` file, if one is provided.

If database information is provided, all dataframes used for analysis are uploaded to it. We use [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server/sql-server-downloads) by default but any kind of database is supported.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
DB_TYPE = os.getenv("DB_TYPE", "sqlserver")
DB_USER = os.getenv("DB_USER", "sqladmin")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "1433")
DB_NAME = os.getenv("DB_NAME", "dataframes")
DB_DRIVER = os.getenv("DB_DRIVER") # some databases require a database driver

### Presentation

By default, Pandas dataframes are truncated when they are printed. We want to be able to view all of the data at once, so we embed the dataframe in a scrollable element.

In [ ]:
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

def custom_scrollable_display(df: pd.DataFrame, max_height=400):
    """
    Custom display function to render DataFrames as scrollable elements.
    
    Parameters:
    - df: The DataFrame to display.
    - max_height: The maximum height of the scrollable area in pixels.
    """
    style = f"""
    <style>
    .scrollable-dataframe {{
        display: inline-block;
        white-space: nowrap;
        overflow-x: scroll;
        max-height: {max_height}px;
        overflow-y: scroll;
    }}
    </style>
    """
    display(HTML(style + f'<div class="scrollable-dataframe">{df.to_html()}</div>'))

def custom_display_hook(df):
    custom_scrollable_display(df)
    return ""

# hook up the custom display function to the automatic printer
InteractiveShell.instance().display_formatter.formatters['text/html'].for_type(pd.DataFrame, custom_display_hook);


### Pre-Commit Hooks (Developer Only)

This notebook uses `nbstripout` to strip notebook output from Git commits. If you are committing code, please run the following command to set up the Git filter.

Poetry is required for the pre-commit hooks, so make sure it is installed before you commit code. You will also need to add the plugin `poetry-plugin-export` in order to run the export hook.
```bash
poetry self add poetry-plugin-export
```

In [ ]:
!nbstripout --install
!pre-commit install

## 1. Data Collection

### Fetch Player Data from NBA API

`nba_api` provides static player and team information, which we will download here so that we can reuse it without requesting the API unnecessarily.

In [ ]:
from nba_api.stats.static import players, teams
PLAYERS_LIST_FILE = "../data/players_list.csv"
TEAMS_LIST_FILE = "../data/teams_list.csv"

if os.path.exists(PLAYERS_LIST_FILE):
    players_list = pd.read_csv(PLAYERS_LIST_FILE)
else:
    players_list = pd.DataFrame(players.get_players())
    players_list.to_csv(PLAYERS_LIST_FILE)

if os.path.exists(TEAMS_LIST_FILE):
    teams_list = pd.read_csv(TEAMS_LIST_FILE)
else:
    teams_list = pd.DataFrame(teams.get_teams())
    teams_list.to_csv(TEAMS_LIST_FILE)


### Fetch Game Data



We're only interested in games that are either in the regular season or in the playoffs. We'll add an enum to distinguish the type of game and use it to differentiate them.

In [ ]:
from enum import Enum
class SeasonType(Enum):
    PRESEASON = 1
    REGULAR_SEASON = 2
    ALL_STAR = 3
    PLAYOFFS = 4
    PLAY_IN = 5
    NBA_CUP = 6
class Season():
    def __init__(self, season_id: int) -> None:
        season_id_str = str(season_id)
        self.season_type = SeasonType(int(season_id_str[0]))
        self.season_year = int(season_id_str[1:])

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import LeagueIDNullable, LeagueID
START_SEASON = 2012
END_SEASON = 2023
GAMES_LIST_FILE = "../data/games_list.csv"
if os.path.exists(GAMES_LIST_FILE):
    games_list: pd.DataFrame = pd.read_csv(GAMES_LIST_FILE)
else:
    games_list = pd.DataFrame()
    for season in range(START_SEASON, END_SEASON + 1):
        # put season into the correct form e.g. 2023 -> 2023-24
        season_str = f"{season}-{str(season + 1)[2:]}"
        print(f"Fetching games for season: {season_str}", end="\r")
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season_str, league_id_nullable=LeagueIDNullable.nba)
        games = gamefinder.get_data_frames()[0]
        games_list = pd.concat([games_list, games], ignore_index=True)
        time.sleep(0.6)
    games_list.to_csv(GAMES_LIST_FILE, index=False)
# games_list["SEASON_ID"].unique()
games_list.head()

### Fetch Play by Plays

In [ ]:
from nba_api.stats.endpoints import playbyplayv3
from requests.exceptions import ReadTimeout
# took 483 minutes to download up to 2012
PBP_LIST_FILE = "../data/pbp_list.csv"
if os.path.exists(PBP_LIST_FILE):
    pbp_list = pd.read_csv(PBP_LIST_FILE)
else:
    unique_games_list = games_list.drop_duplicates(subset="GAME_ID")
    pbp_list = pd.DataFrame()
    for index, row in unique_games_list.iterrows():
        err = False
        game_id = row["GAME_ID"]
        game_date = row["GAME_DATE"]
        season_id = row["SEASON_ID"]
        season = Season(season_id)
        if season.season_type != SeasonType.REGULAR_SEASON and season.season_type != SeasonType.PLAYOFFS:
            continue
        print(f"Fetching play by play for game {game_id} on {game_date}", end="\r")
        while True:
            try:
                pbpfinder = playbyplayv3.PlayByPlayV3(f"{game_id:010}")
                break
            except ReadTimeout as e:
                print(f"{e}! Try again")
            except Exception:
                with open("../data/err.log", "a") as f:
                    print(f"{game_id} does not have a play by play", file=f)
                err = True
                break
        if err:
            continue
        pbp = pbpfinder.get_data_frames()[0]
        pbp_list = pd.concat([pbp_list, pbp], ignore_index=True)
        time.sleep(0.6)
    pbp_list.to_csv(PBP_LIST_FILE, index=False)
pbp_list.head()


### Fetch Boxscores

In [ ]:
from nba_api.stats.endpoints import boxscoretraditionalv3
from nba_api.stats.library.parserv3 import NBAStatsBoxscoreTraditionalParserV3, NBAStatsPlayByPlayParserV3, NBAStatsBoxscoreParserV3

boxscorefinder = boxscoretraditionalv3.BoxScoreTraditionalV3("0041200407")
boxscorefinder.get_data_frames()[0]
# parser = NBAStatsBoxscoreTraditionalParserV3(boxscorefinder.get_dict())
# parser.get_start_bench_data()

## 2. Data Processing

### Unnecessary Columns

Some of this data isn't useful to us, so we'll drop it to ignore the noise.

In [ ]:
columns_to_drop = ["actionNumber", "pointsTotal", "videoAvailable", "actionId", "playerName", "playerNameI", "personId", "teamTricode"]
pbp_list.drop(columns=[col for col in columns_to_drop if col in pbp_list.columns], inplace=True)
pbp_list.head()

### Categorization

To save on memory, we will also turn variables that can be understood as categorical variables into that type.

In [ ]:
categorical_columns = ["gameId", "teamId", "shotResult", "isFieldGoal", "location", "actionType", "subType"]
pbp_list[categorical_columns] = pbp_list[categorical_columns].astype("category")
pbp_list.head()

### Clock

We'll transform the clock data from a string into the total number of seconds.

In [ ]:
if pbp_list["clock"].dtype != "int64":
    pbp_list["clock"] = pbp_list["clock"].astype(str)
    pbp_list["minutes"] = pbp_list["clock"].str[2:4].astype(int)
    pbp_list["seconds"] = pbp_list["clock"].str[5:7].astype(int)
    pbp_list["clock"] = pbp_list["minutes"] * 60 + pbp_list["seconds"]
    pbp_list.drop(columns=["minutes", "seconds"], inplace=True)
pbp_list["clock"].head()

## 3. Data Analysis

### The Clutch

The clutch is one of the most important time periods in an NBA game. Colloquially, it refers to the part of the game where actions taken have a much greater chance of affecting the outcome of the game. As defined by the NBA, this is in the last five minutes of the game when there is a score margin of five or less.

Our analysis will cover how shot distance in the clutch is different from shot distance in general, and how shots taken at a particular distance affect the outcome of the game.

Our analysis will be focused on when shots with assists are being made, so we will isolate these actions for the relevant data.

In [ ]:
substitutions = pbp_list.loc[pbp_list["actionType"] == "Substitution"].reset_index(drop=True)
substitutions.head()

In [ ]:
all_shots = pbp_list.loc[pbp_list["shotResult"].notna()].reset_index(drop=True)
all_shots.head()

We're going to need to calculate the score margin of each shot to determine whether they are in the clutch. However, the score margin is not defined for missed shots, only made shots. We will forward fill the res

In [ ]:
all_shots.loc[all_shots["shotResult"] == "Missed", ["scoreHome", "scoreAway"]] = all_shots.loc[all_shots["shotResult"] == "Missed", ["scoreHome", "scoreAway"]].replace(0, np.nan)
all_shots[['scoreHome', 'scoreAway']] = all_shots.groupby('gameId', observed=True)[['scoreHome', 'scoreAway']].ffill()
all_shots.loc[all_shots["shotResult"] == "Missed", ["scoreHome", "scoreAway"]] = all_shots.loc[all_shots["shotResult"] == "Missed", ["scoreHome", "scoreAway"]].replace(np.nan, 0)
all_shots.head()

We can separate these shots into those that are missed and made.

In [ ]:
made_shots = all_shots.loc[all_shots["shotResult"] == "Made"]
missed_shots = all_shots.loc[all_shots["shotResult"] == "Missed"]
made_shots.head()

We will further isolate a dataframe that contains only shots in the clutch.

In [ ]:
def is_clutch(df):
    is_last_period = df["period"] >= 4
    is_last_five_minutes = df["clock"] <= 300
    is_score_margin_below_five = abs(df["scoreHome"] - df["scoreAway"]) <= 5
    return is_last_period & is_last_five_minutes & is_score_margin_below_five

clutch_shots = all_shots[is_clutch(all_shots)]
clutch_made_shots = made_shots[is_clutch(made_shots)]
clutch_missed_shots = missed_shots[is_clutch(missed_shots)]

print(len(clutch_shots), len(clutch_made_shots), len(clutch_missed_shots))
len(all_shots), len(made_shots), len(missed_shots)

In [ ]:
# Create subplots: 1 row, 2 columns
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

bin_edges = np.histogram_bin_edges(
    np.concatenate([
        all_shots["shotDistance"], made_shots["shotDistance"], missed_shots["shotDistance"],
        clutch_shots["shotDistance"], clutch_made_shots["shotDistance"], clutch_missed_shots["shotDistance"]
    ]),
    bins=30
)

# Histogram for general shots
axes[0].hist(all_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='All Shots')
axes[0].hist(made_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='Made Shots')
axes[0].hist(missed_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='Missed Shots')
axes[0].set_xlabel('Shot Distance')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Shot Distances (General Shots)')
axes[0].legend()

# Histogram for clutch shots
axes[1].hist(clutch_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='Clutch Shots')
axes[1].hist(clutch_made_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='Clutch Made Shots')
axes[1].hist(clutch_missed_shots["shotDistance"], bins=bin_edges, alpha=0.5, label='Clutch Missed Shots')
axes[1].set_xlabel('Shot Distance')
axes[1].set_title('Distribution of Shot Distances (Clutch Shots)')
axes[1].legend()

# Adjust layout
# plt.tight_layout()
plt.show()

In [ ]:
# Compute the combined range for KDE plots
shot_distances = np.concatenate([
    all_shots["shotDistance"], made_shots["shotDistance"], missed_shots["shotDistance"],
    clutch_shots["shotDistance"], clutch_made_shots["shotDistance"], clutch_missed_shots["shotDistance"]
])
x_range = (shot_distances.min(), shot_distances.max())

# Create subplots: 1 row, 2 columns
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# KDE plot for general shots
sns.kdeplot(data=all_shots, x="shotDistance", ax=axes[0], label='All Shots', fill=True)
sns.kdeplot(data=made_shots, x="shotDistance", ax=axes[0], label='Made Shots', fill=True)
sns.kdeplot(data=missed_shots, x="shotDistance", ax=axes[0], label='Missed Shots', fill=True)
axes[0].set_xlim(x_range)
axes[0].set_xlabel('Shot Distance')
axes[0].set_ylabel('Density')
axes[0].set_title('KDE of Shot Distances (General Shots)')
axes[0].legend()

# KDE plot for clutch shots
sns.kdeplot(data=clutch_shots, x="shotDistance", ax=axes[1], label='Clutch Shots', fill=True)
sns.kdeplot(data=clutch_made_shots, x="shotDistance", ax=axes[1], label='Clutch Made Shots', fill=True)
sns.kdeplot(data=clutch_missed_shots, x="shotDistance", ax=axes[1], label='Clutch Missed Shots', fill=True)
axes[1].set_xlim(x_range)
axes[1].set_xlabel('Shot Distance')
axes[1].set_title('KDE of Shot Distances (Clutch Shots)')
axes[1].legend()

# Adjust layout
plt.tight_layout()

In [ ]:
# Create subplots: 1 row, 2 columns
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Scatter plot for general shots with regression lines
sns.regplot(data=all_shots, x="shotDistance", y=all_shots.index, ax=axes[0], scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
sns.regplot(data=made_shots, x="shotDistance", y=made_shots.index, ax=axes[0], scatter_kws={'alpha':0.5}, line_kws={'color':'blue'})
sns.regplot(data=missed_shots, x="shotDistance", y=missed_shots.index, ax=axes[0], scatter_kws={'alpha':0.5}, line_kws={'color':'green'})
axes[0].set_xlabel('Shot Distance')
axes[0].set_ylabel('Index')
axes[0].set_title('Shot Distance vs. Index (General Shots)')
axes[0].legend(['All Shots', 'Made Shots', 'Missed Shots'], loc="upper right")

# Scatter plot for clutch shots with regression lines
sns.regplot(data=clutch_shots, x="shotDistance", y=clutch_shots.index, ax=axes[1], scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
sns.regplot(data=clutch_made_shots, x="shotDistance", y=clutch_made_shots.index, ax=axes[1], scatter_kws={'alpha':0.5}, line_kws={'color':'blue'})
sns.regplot(data=clutch_missed_shots, x="shotDistance", y=clutch_missed_shots.index, ax=axes[1], scatter_kws={'alpha':0.5}, line_kws={'color':'green'})
axes[1].set_xlabel('Shot Distance')
axes[1].set_title('Shot Distance vs. Index (Clutch Shots)')
axes[1].legend(['Clutch Shots', 'Clutch Made Shots', 'Clutch Missed Shots'], loc="upper right")

# Adjust layout
plt.tight_layout()
plt.show()


## 4. Data Reporting

To create our final report from our analysis, we will be using Power BI. We have a Microsoft SQL Server database that our Power BI report will import the tables from. If no database is available, the dataframes will instead export as an Excel spreadsheet, which can be manually uploaded to Power BI.

In [ ]:
# from pandas import ExcelWriter
# import sqlalchemy

# dfs = {"assists_in_the_clutch": df}
# DB_PASSWORD = None

# def upload_dataframes(dfs: Dict[str, pd.DataFrame]) -> None:
#     inspector = sqlalchemy.inspect(engine)
#     existing_tables = inspector.get_table_names()

#     for table_name, df in dfs.items():
#         # check if table already exists
#         if table_name in existing_tables:
#             # if there are no changes to the table, do not write to it
#             existing_df = pd.read_sql_table(table_name, engine)
#             if df.shape == existing_df.shape and df.equals(existing_df):
#                 print(f"No changes detected for table {table_name}. Skipping upload.")
#                 continue
#         else:
#             print(f"Table {table_name} does not exist. Creating a new one.")
#         df.to_sql(table_name, engine, if_exists="replace", index=False)
#         print(f"Uploaded dataframe to table {table_name}.")

# if DB_PASSWORD:
#     DATABASE_URL = f"{DB_TYPE}://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}{'?driver=' if DB_DRIVER else ''}{DB_DRIVER.replace(' ', '+') if DB_DRIVER else ''}"

#     engine = sqlalchemy.create_engine(DATABASE_URL)

#     with engine.connect() as connection:
#         print("Connection to the database was successful!")

#     upload_dataframes(dfs)
# else:
#     EXCEL_MAX_ROWS = 1048575
#     excel_writer = pd.ExcelWriter("../data/dataframes.xlsx")
#     for name, df in dfs.items():
#         df.head(EXCEL_MAX_ROWS).to_excel(excel_writer, sheet_name=name)
